In [1]:
#v2
#in v3, we are going to keep track of time duration per transition
##expanded version - edit 3/3 - might try to retain team specific info for time distributions

import pandas as pd
from helper_functions.game_states_logic import get_play_states
from helper_functions.pull_game_logic import pull_game
import os
from tqdm import tqdm
from collections import Counter
import numpy as np

In [2]:
team_database = pd.read_excel('Team_Names_Abbrs_v02.xlsx')

In [3]:
# existing_transition_times = pd.read_excel('All_Transition_With_Times_Not_Aggregated.xlsx')

In [4]:
for filename in os.listdir('raw_game_pulls'):
    teams = filename.split('for ')[1].split('.xlsx')[0]
    team1, team2 = teams.split(' vs ')
    reduce_df = team_database[team_database['name']==team1].copy()
    if len(reduce_df) > 1:
        print('Collision with {}'.format(team1))
    elif len(reduce_df) == 0:
        print('No match for {}'.format(team1))

    reduce_df = team_database[team_database['name']==team2].copy()
    if len(reduce_df) > 1:
        print('Collision with {}'.format(team2))
    elif len(reduce_df) == 0:
        print('No match for {}'.format(team2))

In [5]:
xs=[]
ys=[]

num_games = 0

In [6]:
#initialize full dataframe
#changing this on 2/26 - speed up by making it a dict or list
# final_df = pd.DataFrame()
df_dict = dict()

#iterate through files
for filename in tqdm(os.listdir('raw_game_pulls')):

    #initialize individual game dataframe
    game_tracking_df = pd.DataFrame()

    #parse filename
    teams = filename.split('for ')[1].split('.xlsx')[0]
    team1, team2 = teams.split(' vs ')
    team_A = team_database[team_database['name']==team1]['abbreviations'].values[0]
    team_B = team_database[team_database['name']==team2]['abbreviations'].values[0]

    #pull file data
    current_data = pd.read_excel('raw_game_pulls/{}'.format(filename))

    #pull states
    all_states = get_play_states(current_data, team_A, team_B)
    final_game = pd.DataFrame()
    for i in range(len(all_states)):
        half_tuples = all_states[i]

        # just take play, not timestamp
        smaller_list = [x for x in half_tuples if x[0] != 'UNNEC']

        transition_and_time_elapsed = []

        for j in range(len(smaller_list)-1):
            tran = (smaller_list[j][0], smaller_list[j+1][0])
            timestamp_1 = smaller_list[j][1].split(':')
            timestamp_2 = smaller_list[j+1][1].split(':')

            seconds_1 = int(timestamp_1[0]) * 60 + int(timestamp_1[1])
            seconds_2 = int(timestamp_2[0]) * 60 + int(timestamp_2[1])
            time_elapsed = seconds_1-seconds_2
            datum = (tran, time_elapsed)
            transition_and_time_elapsed.append(datum)


        #that means rather than this logic, we should count the transitions in here

        curr_half = pd.DataFrame(transition_and_time_elapsed, columns = ['Transition','Time'])
        curr_half['Period']=i+1
        final_game = pd.concat([final_game,curr_half])

        list_of_pairs = [tuple(smaller_list[j:j+2]) for j in range(len(smaller_list)-2)]

        # curr_half = pd.DataFrame({'Transition': list_of_pairs})
        # curr_half['Period']=i+1
        game_tracking_df=pd.concat([game_tracking_df,curr_half]).reset_index(drop=True)
    game_tracking_df['filename']=filename
    # final_df=pd.concat([final_df,game_tracking_df]).reset_index(drop=True)
    df_dict[filename] = game_tracking_df
final_df = pd.concat(list(df_dict.values()))

  0%|          | 0/5497 [00:00<?, ?it/s]

100%|██████████| 5497/5497 [03:46<00:00, 24.24it/s]


In [7]:
rel_data = final_df[['Transition','Time','filename']].copy().reset_index(drop=True)

In [8]:
#v4 too big :(

In [9]:
rel_data.to_csv('All_Transition_With_Times_Not_Aggregated_v05.csv',index=False)